Grundsätzlich: 
    12009: Wirkleistung
    12008: Blindleistung

eiot IDs:
UW Bollingstedt Süd Feld E42: 
    SHNG_E:DSO:SHN:ST:1000364072:SA:110:SF:E42:MSZ::MW:12008    c076ad04-1a18-434d-8e86-34e081b0e4f6
    SHNG_E:DSO:SHN:ST:1000364072:SA:110:SF:E42:MSZ::MW:12009    102fa684-58ca-451d-a0ae-6341accc6a36



UW Tarp Feld E01:
Blindleistung ist nicht gegeben (warum auch immer)
    SHNG_E:DSO:SHN:ST:STUW45121:SA:110:SF:E01:MSZ::MW:P     03636523-35f2-428f-86e8-c260fafbe330
    SHNG_E:DSO:SHN:ST:STUW45121:SA:110:SF:E01:MSZ::MW:IS    a8b25505-1352-47a1-9798-4c0641cf3302
    SHNG_E:DSO:SHN:ST:STUW45121:SA:110:SF:E01:MSZ::MW:U     600fd8d0-e572-4aff-a2ac-7f2b02c18066




UW Schuby West Feld E24
    SHNG_E:DSO:SHN:ST:STUW47570:SA:110:SF:E24:MSZ::MW:12008
    SHNG_E:DSO:SHN:ST:STUW47570:SA:110:SF:E24:MSZ::MW:12009


UW Jübek Ost Feld E01:
    Kann ignoriert werden, da es sich hier um einen Erder handelt..


In [ ]:
import numpy as np
import pandas as pd

# =========================
# definition of paths:
# Achte auf die Windows Falle; also einlesen der Pfade mit r"..."
UW_Boll_P = r"Daten\CSV_Raw\UW Bollingstedt E42 P und Q\SHNG_E_DSO_SHN_ST_1000364072_SA_110_SF_E42_MSZ__MW_12008_20251001.csv"
UW_Boll_Q = r"Daten\CSV_Raw\UW Bollingstedt E42 P und Q\SHNG_E_DSO_SHN_ST_1000364072_SA_110_SF_E42_MSZ__MW_12009_20251001.csv"

UW_Tarp_IS = r"Daten\CSV_Raw\UW Tarp E01 P IS und U\SHNG_E_DSO_SHN_ST_STUW45121_SA_110_SF_E01_MSZ__MW_IS_20251001.csv"
UW_Tarp_U  = r"Daten\CSV_Raw\UW Tarp E01 P IS und U\SHNG_E_DSO_SHN_ST_STUW45121_SA_110_SF_E01_MSZ__MW_U_20251001.csv"

UW_Schuby_P = r"Daten\CSV_Raw\UW Schuby West\SHNG_E_DSO_SHN_ST_STUW47570_SA_110_SF_E24_MSZ__MW_12008_20251001.csv"
UW_Schuby_Q = r"Daten\CSV_Raw\UW Schuby West\SHNG_E_DSO_SHN_ST_STUW47570_SA_110_SF_E24_MSZ__MW_12009_20251001.csv"
# =========================


SQRT3 = np.sqrt(3.0)

def S_from_P_Q(P_MW, Q_MVAr, signed=True):
    """
    Berechnet S aus P und Q.
    Wenn signed=True, wird das Vorzeichen von P übernommen.
    """
    P = pd.Series(P_MW, dtype="float64")
    Q = pd.Series(Q_MVAr, dtype="float64")
    S_abs = np.sqrt(P.pow(2) + Q.pow(2))
    if signed:
        return np.sign(P) * S_abs
    else:
        return S_abs


def S_from_U_IS(U_kV, I_A, signed=True):
    """
    Berechnet S aus U [kV] und I [A] (Dreiphasen).
    Ergebnis in MVA.
    Wenn signed=True, wird das Vorzeichen von I übernommen.
    """
    U = pd.Series(U_kV, dtype="float64")
    I = pd.Series(I_A, dtype="float64") / 1000.0  # A → kA
    S_abs = SQRT3 * U.abs() * I.abs()
    if signed:
        return np.sign(I) * S_abs
    else:
        return S_abs



# CSVs einlesen (immer nur 2 Spalten, Timestamp + Wert)
df_boll_p = pd.read_csv(UW_Boll_P, sep=",", header=0)
df_boll_q = pd.read_csv(UW_Boll_Q, sep=",", header=0)

df_tarp_is = pd.read_csv(UW_Tarp_IS, sep=",", header=0)
df_tarp_u  = pd.read_csv(UW_Tarp_U,  sep=",", header=0)

df_schuby_p = pd.read_csv(UW_Schuby_P, sep=",", header=0)
df_schuby_q = pd.read_csv(UW_Schuby_Q, sep=",", header=0)

# Scheinleistung berechnen
S_boll = S_from_P_Q(df_boll_p.iloc[:,1], df_boll_q.iloc[:,1])
S_tarp = S_from_U_IS(df_tarp_u.iloc[:,1], df_tarp_is.iloc[:,1])
S_schuby = S_from_P_Q(df_schuby_p.iloc[:,1], df_schuby_q.iloc[:,1])

# Ergebnisse in DataFrames mit Zeitstempel zusammenführen
df_boll_result = pd.DataFrame({
    "timestamp": pd.to_datetime(df_boll_p.iloc[:,0]),
    "S_MVA": S_boll
})

df_tarp_result = pd.DataFrame({
    "timestamp": pd.to_datetime(df_tarp_u.iloc[:,0]),
    "S_MVA": S_tarp
})

df_schuby_result = pd.DataFrame({
    "timestamp": pd.to_datetime(df_schuby_p.iloc[:,0]),
    "S_MVA": S_schuby
})


# -----------------------------
# 1) Merge auf gemeinsame Zeitbasis
# -----------------------------
df_merge = (
    df_boll_result[["timestamp", "S_MVA"]]
    .merge(df_tarp_result[["timestamp", "S_MVA"]], on="timestamp", how="inner", suffixes=("_Boll", "_Tarp"))
    .merge(df_schuby_result[["timestamp", "S_MVA"]], on="timestamp", how="inner")
    .rename(columns={"S_MVA": "S_Schuby"})
    .sort_values("timestamp")
    .reset_index(drop=True)
)

# Wenn du bereits signiertes S in den Einzel-DFs berechnet hast (Spalte 'S_signed'),
# kannst du die folgenden optionalen Ersetzungen aktivieren:
if "S_signed" in df_boll_result.columns:
    df_merge["S_MVA_Boll"] = df_boll_result.set_index("timestamp").loc[df_merge["timestamp"], "S_signed"].to_numpy()
if "S_signed" in df_tarp_result.columns:
    df_merge["S_MVA_Tarp"] = df_tarp_result.set_index("timestamp").loc[df_merge["timestamp"], "S_signed"].to_numpy()
if "S_signed" in df_schuby_result.columns:
    df_merge["S_Schuby"] = df_schuby_result.set_index("timestamp").loc[df_merge["timestamp"], "S_signed"].to_numpy()

# -----------------------------
# 2) ΔS bilden
# -----------------------------
df_merge["dS_Boll"]   = df_merge["S_MVA_Boll"].diff()
df_merge["dS_Tarp"]   = df_merge["S_MVA_Tarp"].diff()
df_merge["dS_Schuby"] = df_merge["S_Schuby"].diff()

# Erste Zeile fällt wegen diff() raus
df_merge = df_merge.dropna(subset=["dS_Boll", "dS_Tarp", "dS_Schuby"]).reset_index(drop=True)

# -----------------------------
# 3) Spikes in Boll erkennen
#    Variante A: fixer ΔS-Schwellenwert (einfach)
# -----------------------------
threshold = 30.0  # MVA – an deine Daten anpassen
spike_idx = df_merge.index[df_merge["dS_Boll"].abs() > threshold].to_list()


print(f"Anzahl erkannter Spikes in Boll: {len(spike_idx)}")

# -----------------------------
# 4) Fenster um Spikes sammeln (um Reaktion in Nachbarn mitzunehmen)
# -----------------------------
window = 2  # z.B. die 2 Punkte davor und danach mitnehmen
use_rows = set()
for i in spike_idx:
    for j in range(max(0, i - window), min(len(df_merge), i + window + 1)):
        use_rows.add(j)
use_rows = sorted(use_rows)

df_spikes = df_merge.iloc[use_rows].copy()

# -----------------------------
# 5) Korrelation NUR auf Spike-Phasen (ΔS)
# -----------------------------
corr_matrix = df_spikes[["dS_Boll", "dS_Tarp", "dS_Schuby"]].corr(method="pearson")
print("\nKorrelationsmatrix (nur Spike-Phasen, auf ΔS):")
print(corr_matrix)

corr_boll_tarp   = corr_matrix.loc["dS_Boll", "dS_Tarp"]
corr_boll_schuby = corr_matrix.loc["dS_Boll", "dS_Schuby"]

print(f"\nKorrelation ΔBoll–ΔTarp (Spikes):   {corr_boll_tarp:.3f}")
print(f"Korrelation ΔBoll–ΔSchuby (Spikes): {corr_boll_schuby:.3f}")

if abs(corr_boll_tarp) > abs(corr_boll_schuby):
    print("👉 Bei Spikes hängt Boll stärker mit Tarp zusammen.")
else:
    print("👉 Bei Spikes hängt Boll stärker mit Schuby zusammen.")


import statsmodels.api as sm

# --- Regression ΔTarp ~ ΔBoll ---
X = df_spikes["dS_Boll"]
X = sm.add_constant(X)  # Bias/Intercept erlauben
y = df_spikes["dS_Tarp"]

model_tarp = sm.OLS(y, X).fit()
print("\nRegression ΔTarp ~ ΔBoll")
print(model_tarp.summary())

# Sensitivitätsfaktor (Steigung):
a_tarp = model_tarp.params["dS_Boll"]
print(f"Sensitivität Tarp: ΔTarp ≈ {a_tarp:.2f} * ΔBoll")

# --- Regression ΔSchuby ~ ΔBoll ---
X = df_spikes["dS_Boll"]
X = sm.add_constant(X)
y = df_spikes["dS_Schuby"]

model_schuby = sm.OLS(y, X).fit()
print("\nRegression ΔSchuby ~ ΔBoll")
print(model_schuby.summary())

b_schuby = model_schuby.params["dS_Boll"]
print(f"Sensitivität Schuby: ΔSchuby ≈ {b_schuby:.2f} * ΔBoll")


